# Pattern Plot Generator

Generate power consumption plots for specific dates and times.

## Setup

In [ ]:
# Install required packages
!pip install pandas plotly -q

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Configuration - UPDATE THESE PATHS
EXPERIMENT_PATH = '/content/drive/MyDrive/YOUR_EXPERIMENT_FOLDER'  # Path to experiment output
# Example: '/content/drive/MyDrive/exp006_partial_matching_20260126_210435'

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
from pathlib import Path
import os

def load_summarized_data(experiment_dir, house_id, run_number=0):
    """Load summarized data for a house."""
    # Try different path structures
    paths_to_try = [
        Path(experiment_dir) / f'house_{house_id}' / f'run_{run_number}' / f'house_{house_id}' / 'summarized',
        Path(experiment_dir) / f'run_{run_number}' / f'house_{house_id}' / 'summarized',
        Path(experiment_dir) / f'house_{house_id}' / 'summarized',
    ]
    
    for summarized_dir in paths_to_try:
        if summarized_dir.exists():
            files = sorted(summarized_dir.glob(f'summarized_{house_id}_*.csv'))
            if files:
                dfs = []
                for f in files:
                    df = pd.read_csv(f)
                    df['timestamp'] = pd.to_datetime(df['timestamp'], format='mixed', dayfirst=True)
                    dfs.append(df)
                print(f'Loaded {len(files)} files from {summarized_dir}')
                return pd.concat(dfs, ignore_index=True)
    
    print(f'No data found for house {house_id}')
    return None

def calculate_y_range(df, phases):
    """Calculate unified y-axis range."""
    y_min, y_max = float('inf'), float('-inf')
    for phase in phases:
        for col in [f'original_{phase}', f'remaining_{phase}']:
            if col in df.columns:
                y_min = min(y_min, df[col].min())
                y_max = max(y_max, df[col].max())
    padding = (y_max - y_min) * 0.05
    return (y_min - padding, y_max + padding)

print('Functions loaded!')

## Generate Plot

Enter the house ID, date, and time window you want to visualize.

In [ ]:
# CONFIGURATION - Change these values
HOUSE_ID = '140'           # House ID
DATE = '2021-05-15'        # Date (YYYY-MM-DD)
CENTER_TIME = '14:30'      # Center time (HH:MM)
HOURS_BEFORE = 2           # Hours to show before center time
HOURS_AFTER = 2            # Hours to show after center time

In [ ]:
# Load data
data = load_summarized_data(EXPERIMENT_PATH, HOUSE_ID)

if data is not None:
    # Parse center time
    date_obj = datetime.strptime(DATE, '%Y-%m-%d')
    time_parts = CENTER_TIME.split(':')
    center = date_obj.replace(hour=int(time_parts[0]), minute=int(time_parts[1]))
    
    # Create time window
    window_start = center - timedelta(hours=HOURS_BEFORE)
    window_end = center + timedelta(hours=HOURS_AFTER)
    
    # Filter data
    mask = (data['timestamp'] >= window_start) & (data['timestamp'] <= window_end)
    window_data = data[mask].copy()
    
    print(f'Found {len(window_data)} data points in window')
    print(f'Time range: {window_start} to {window_end}')

In [ ]:
# Create plot
if data is not None and len(window_data) > 0:
    phases = ['w1', 'w2', 'w3']
    phase_colors = {'w1': '#1f77b4', 'w2': '#2ca02c', 'w3': '#ff7f0e'}
    
    # Check what data we have
    has_remaining = any(f'remaining_{p}' in window_data.columns for p in phases)
    n_rows = 2 if has_remaining else 1
    row_titles = ['Original Power', 'Remaining Power'] if has_remaining else ['Power']
    
    # Create subplots
    fig = make_subplots(
        rows=n_rows, cols=3,
        shared_xaxes=True,
        vertical_spacing=0.1,
        horizontal_spacing=0.05,
        subplot_titles=[f'Phase {p.upper()}' for p in phases] + [''] * 3,
        row_titles=row_titles
    )
    
    for col_idx, phase in enumerate(phases, 1):
        color = phase_colors[phase]
        
        # Original power
        if f'original_{phase}' in window_data.columns:
            fig.add_trace(
                go.Scatter(
                    x=window_data['timestamp'],
                    y=window_data[f'original_{phase}'],
                    mode='lines',
                    name=f'Original {phase.upper()}',
                    line=dict(color=color, width=1.5),
                    showlegend=(col_idx == 1)
                ),
                row=1, col=col_idx
            )
        
        # Remaining power
        if has_remaining and f'remaining_{phase}' in window_data.columns:
            fig.add_trace(
                go.Scatter(
                    x=window_data['timestamp'],
                    y=window_data[f'remaining_{phase}'],
                    mode='lines',
                    name=f'Remaining {phase.upper()}',
                    line=dict(color=color, width=1.5),
                    showlegend=(col_idx == 1)
                ),
                row=2, col=col_idx
            )
        
        # Highlight center time
        for row in range(1, n_rows + 1):
            fig.add_vline(x=center, line_dash='dash', line_color='red', row=row, col=col_idx)
    
    # Unified Y-axis
    y_min, y_max = calculate_y_range(window_data, phases)
    for row in range(1, n_rows + 1):
        for col in range(1, 4):
            fig.update_yaxes(range=[y_min, y_max], row=row, col=col)
    
    # Layout
    fig.update_layout(
        title=f'House {HOUSE_ID} - {DATE} @ {CENTER_TIME}',
        height=300 * n_rows,
        showlegend=True,
        legend=dict(orientation='h', y=1.02),
        hovermode='x unified',
        template='plotly_white'
    )
    
    fig.show()
else:
    print('No data to plot')

## Generate Multiple Plots (from date list)

Copy dates from the HTML report and generate plots for all of them.

In [ ]:
# CONFIGURATION
DATES = ['2021-05-05', '2021-05-06', '2021-05-07']  # List of dates
CENTER_TIME = '14:30'
HOURS_BEFORE = 1
HOURS_AFTER = 1

# Generate plots for each date
if data is not None:
    for date_str in DATES:
        print(f'\n=== {date_str} ===')
        
        date_obj = datetime.strptime(date_str, '%Y-%m-%d')
        time_parts = CENTER_TIME.split(':')
        center = date_obj.replace(hour=int(time_parts[0]), minute=int(time_parts[1]))
        
        window_start = center - timedelta(hours=HOURS_BEFORE)
        window_end = center + timedelta(hours=HOURS_AFTER)
        
        mask = (data['timestamp'] >= window_start) & (data['timestamp'] <= window_end)
        window_data = data[mask].copy()
        
        if len(window_data) == 0:
            print(f'No data for {date_str}')
            continue
        
        # Create simple plot (same code as above, simplified)
        phases = ['w1', 'w2', 'w3']
        fig = make_subplots(rows=2, cols=3, shared_xaxes=True,
                           subplot_titles=[f'Phase {p.upper()}' for p in phases] + ['', '', ''])
        
        for col_idx, phase in enumerate(phases, 1):
            if f'original_{phase}' in window_data.columns:
                fig.add_trace(go.Scatter(x=window_data['timestamp'], y=window_data[f'original_{phase}'],
                              mode='lines', name=f'Orig {phase}', showlegend=(col_idx==1)), row=1, col=col_idx)
            if f'remaining_{phase}' in window_data.columns:
                fig.add_trace(go.Scatter(x=window_data['timestamp'], y=window_data[f'remaining_{phase}'],
                              mode='lines', name=f'Rem {phase}', showlegend=(col_idx==1)), row=2, col=col_idx)
        
        y_min, y_max = calculate_y_range(window_data, phases)
        for row in range(1, 3):
            for col in range(1, 4):
                fig.update_yaxes(range=[y_min, y_max], row=row, col=col)
        
        fig.update_layout(title=f'House {HOUSE_ID} - {date_str} @ {CENTER_TIME}', height=500, template='plotly_white')
        fig.show()